In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain_huggingface pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [ ]:
!pip install openai

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet langchain-community unstructured openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 995.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 980.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.3 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet  docx2txt

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install langchain_chroma

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "YOUR_LANGCHAIN_API_KEY"
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")



In [ ]:
from docx import Document
import pandas as pd
import re

# # Lists to store structured data
chinese_sections = []
english_sections = []
clean_text = []



def pre_pro(path):
    doc = Document(path)
    # # Iterate through each paragraph in the document

    for p in doc.paragraphs:
        text = p.text.strip()
        if len(text) == 0 :
            continue
        else:
            clean_text.append(text)

    for i, para in enumerate(clean_text):

        patterns = r"[\u4e00-\u9fff]+"
        chinese = re.findall(patterns,para)
        is_chinese = True if len(chinese) > 0 else False
        sup_chinese = True if i%2 == 0 else False
        chinese_nums = -1
        english_nums = -1
        if is_chinese and sup_chinese:
            prev_chinese = re.findall(patterns,clean_text[i-1])
            prev_is_chinese = True if len(prev_chinese) > 0 else False
            prev_sup_chinese = True if (i-1)%2 == 0 else False

            if not prev_is_chinese and  prev_sup_chinese:
                chinese_sections[chinese_nums]+=f" {para}"
            elif prev_is_chinese and not prev_sup_chinese:
                chinese_sections[chinese_nums]+=f" {para}"
            else:
                chinese_sections.append(para)
                chinese_nums+=1
        elif is_chinese and not sup_chinese:
            prev_chinese = re.findall(patterns,clean_text[i-1])
            prev_is_chinese = True if len(prev_chinese) > 0 else False
            prev_sup_chinese = True if (i-1)%2 == 0 else False

            if prev_is_chinese and prev_sup_chinese:
                chinese_sections[chinese_nums]+=f" {para}"
            else:
                chinese_sections.append(para)
                chinese_nums+=1

        elif not is_chinese and not sup_chinese:
            prev_chinese = re.findall(patterns,clean_text[i-1])
            prev_is_chinese = True if len(prev_chinese) > 0 else False
            prev_sup_chinese = True if (i-1)%2 == 0 else False
            if not prev_is_chinese and prev_sup_chinese:
                english_sections[english_nums]+=f" {para}"
            else:
                english_sections.append(para)
                english_nums+=1
        elif not is_chinese and sup_chinese:
            prev_chinese = re.findall(patterns,clean_text[i-1])
            prev_is_chinese = True if len(prev_chinese) > 0 else False
            prev_sup_chinese = True if (i-1)%2 == 0 else False

            if prev_is_chinese and not prev_sup_chinese:
                english_sections.append(para)
            else:
                english_sections[english_nums]+=f" {para}"
        elif is_chinese and not sup_chinese:
            chinese_sections[chinese_nums]+= f" {para}"



    # # Append the last sections if any
    # if current_chinese and current_english:
    #     chinese_sections.append('\n'.join(current_chinese))
    #     english_sections.append('\n'.join(current_english))

    # # Create a DataFrame from the lists
        data = {
            "中文法規": chinese_sections,
            "英文法規": english_sections
        }
    return pd.DataFrame(data)
# # Example usage:
df = pre_pro("regulations.docx")
df["中文法規"].to_csv("output.csv")
# # print(df.head())


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_community.document_loaders.csv_loader import CSVLoader

# Load, chunk and index the contents of the blog.

loader = CSVLoader("output.csv")
docs = loader.load()

docs_filtered = filter_complex_metadata(docs)
vectorstore = Chroma.from_documents(documents=docs_filtered, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")


# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")

In [ ]:
import re

from openai import OpenAI

client = OpenAI()
def CoT(*args):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system",
         "content": "你是一個校園法規翻譯專家，請根據使用者的指令跟需求給予相應的幫助。請仔細且邏輯地思考，並解釋您的答案。"},
        {"role": "user", "content":f"""現在請你幫我根據下列中文法規及對應的英譯法規，推論該英譯法規相應的語言規則、模式、語氣及用詞。請仔細閱讀完，再邏輯地思考，並解釋您的答案，以繁體中文輸出。\n法規：\n{args}"""}
      ]
    )
    return completion.choices[0].message.content

def translation(rules, regulation):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system",
         "content": "你是一個校園法規翻譯專家，請根據使用者的指令跟需求給予相應的幫助。"},
        {"role": "user", "content":f"""現在請你幫我根據下列法規翻譯規則、模式、語氣及用詞，將以下的中文法規翻譯出相應且簡潔直接的英譯版本。輸出格式如下：中文法規 \n 英文法規。 無需給我其他不相干的說明及敘述。\n 法規翻譯規則及模式：\n{rules}\n中文法規：\n{regulation}"""}
      ]
    )
    return completion.choices[0].message.content


# def proofread(regulation):
#     completion = client.chat.completions.create(
#       model="gpt-4o",
#       messages=[
#         {"role": "system",
#          "content": "You are a professional University legislative committ member. Please help users according to their requests and instructions."},
#         {"role": "user", "content":f"""Please proofread the following regulation with native, legal, professional and cohesive tones. Do not give me other irrevant and redundant response, just give me the result of proofreaded regulation. \n Regulation: \n{regulation}"""}
#       ]
#     )
#     return completion.choices[0].message.content

def main(regulation):
    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8})
    retrieved_docs = retriever.invoke(regulation)

    if len(retrieved_docs) > 3:
        regulations = ["".join(re.findall(r"\d+\n中文法規\:\s(.+)", i.page_content)) for i in retrieved_docs]
    elif len(retrieved_docs) <= 3:
        retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5})
        retrieved_docs = retriever.invoke(regulation)
        regulations = ["".join(re.findall(r"\d+\n中文法規\:\s(.+)", i.page_content)) for i in retrieved_docs[:2]]
    else:
        retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2})
        retrieved_docs = retriever.invoke(regulation)
        regulations = ["".join(re.findall(r"\d+\n中文法規\:\s(.+)", i.page_content)) for i in retrieved_docs]


    chinese_regus = []
    english_regus = []
    cot = []
    for r in regulations:
        for i, cr in enumerate(df["中文法規"]):
            if r == cr:
                if r in chinese_regus:
                    continue
                else:
                    chinese_regus.append(df["中文法規"][i])
                    english_regus.append(df["英文法規"][i])

    for index in range(len(chinese_regus)):
        cot.append(CoT(chinese_regus[index], english_regus[index]))

    result = translation(cot[0],regulation)
    return regulation, chinese_regus, english_regus, cot[0],result

In [ ]:
translation_result = main("第七條 本獎獲獎者於同學年度不得兼領本校傅鐘獎學金。同學年度本獎、海推獎學金及國際學生助學金僅限領取一種獎助學金，若本獎之獎學金金額高於海推獎學金或國際學生助學金時，由本獎學金補足差額。")

In [ ]:
print(translation_result[2])

['Article 5 Application process: New students: Those who meet the qualification mentioned above shall apply for the scholarship within two weeks after the UECOCS announces the admission results. The Office of International Affairs shall submit the list of applicants to the International Student Scholarship Evaluation Committee for review and the results of which shall be approved by the President. The scholarship cannot be carried over to the next year. Renewal application: The scholarship recipients shall attach the transcripts of the previous year within one month after the fall semester begins and submit a renewal application. The Office of International Affairs shall submit the list of applicants to the International Student Scholarship Evaluation Committee for review, which shall be approved by the President. The approved number of scholarships in this scholarship case is limited to 20% of the total number of Overseas Chinese Students in the University.', "Article 4 Award System: 

In [ ]:
final_result = pd.DataFrame({"中文待翻法規":a[0],"Chinese_retrievals":a[1],"English_retrievals":a[2],"CoT inferenced rules":a[3],"Translation":a[4]})
final_result.to_excel("result.xlsx")